In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir deep_style_turn

In [0]:
import os
os.chdir("/content/drive/My Drive/data/deep_style_turn")

In [5]:
!pwd

/content/drive/My Drive/data/deep_style_turn


In [25]:
!wget http://n.sinaimg.cn/collect/crawl/20160224/j0or-fxprucs6463696.jpg

--2019-08-16 05:16:13--  http://n.sinaimg.cn/collect/crawl/20160224/j0or-fxprucs6463696.jpg
Resolving n.sinaimg.cn (n.sinaimg.cn)... 2.21.41.13, 2a02:26f0:7b:481::102d, 2a02:26f0:7b:48a::102d
Connecting to n.sinaimg.cn (n.sinaimg.cn)|2.21.41.13|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58368 (57K) [image/jpeg]
Saving to: ‘j0or-fxprucs6463696.jpg’

j0or-fxprucs6463696 100%[===================>]  57.00K  --.-KB/s    in 0.003s  

2019-08-16 05:16:13 (21.5 MB/s) - ‘j0or-fxprucs6463696.jpg’ saved [58368/58368]



In [6]:
from keras.preprocessing.image import load_img, img_to_array

target_image_path = 'portrait.jpg' #想要变换的图像的路径
style_reference_image_path = 'transfer_style_reference.jpg' #风格图像的路径

#生成图像的尺寸
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

Using TensorFlow backend.


- 辅助函数

In [0]:
import numpy as np
from keras.applications import vgg19

In [0]:
def preprocess_image(image_path):
  img = load_img(image_path, target_size=(img_height, img_width))
  img = img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = vgg19.preprocess_input(img)
  return img

def deprocess_image(x):
  #vgg19.preprocess_input 的作用是减去ImageNet 的平均像素值，
  #使其中心为0。这里相当于vgg19.preprocess_input 的逆操作
  x[:,:,0] += 103.939
  x[:,:,1] += 116.779
  x[:,:,2] += 123.68
  #将图像由BGR 格式转换为RGB 格式。这也是vgg19.preprocess_input 逆操作的一部分
  x = x[:,:,::-1]
  
  x = np.clip(x,0,255).astype('uint8')
  return x

下面构建VGG19 网络。它接收三张图像的批量作为输入，三张图像分别是风格参考图像、
目标图像和一个用于保存生成图像的占位符。占位符是一个符号张量，它的值由外部Numpy 张
量提供。风格参考图像和目标图像都是不变的，因此使用K.constant 来定义，但生成图像的
占位符所包含的值会随着时间而改变。

-  加载预训练的VGG19 网络，并将其应用于三张图像

In [0]:
from keras import backend as K

In [13]:
target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
#这个占位符用于保存生成图像
combination_image = K.placeholder((1,img_height,img_width,3))

#将三张图像合并为一个批量
input_tensor = K.concatenate([target_image,
                             style_reference_image,
                             combination_image],
                            axis=0)

#利用三张图像组成的批量作为输入
#来构建VGG19 网络。加载模型将
#使用预训练的ImageNet 权重
model = vgg19.VGG19(input_tensor=input_tensor,
                   weights='imagenet',
                   include_top=False)
print('Model loaded.')

W0816 02:31:55.201576 140092940949376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0816 02:31:55.208717 140092940949376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0816 02:31:55.210487 140092940949376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0816 02:31:55.251318 140092940949376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



80142336/80134624 [==============================] - 3s 0us/step


W0816 02:31:59.449075 140092940949376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0816 02:31:59.450181 140092940949376 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



Model loaded.


我们来定义内容损失，它要保证目标图像和生成图像在VGG19 卷积神经网络的顶层具有相
似的结果。

- 内容损失

In [0]:
def content_loss(base, combination):
  return K.sum(K.square(combination - base))

接下来是风格损失。它使用一个辅助函数来计算输入矩阵的格拉姆矩阵，即原始特征矩阵
中相互关系的映射。
- 风格损失

In [0]:
def gram_matrix(x):
  features = K.batch_flatten(K.permute_dimensions(x,(2,0,1)))
  gram = K.dot(features, K.transpose(features))
  return gram

def style_loss(style, combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  channels = 3
  size = img_height * img_width
  return K.sum(K.square(S - C )) / (4. * (channels ** 2) * (size ** 2 ))

除了这两个损失分量，我们还要添加第三个——总变差损失（total variation loss），它对生成
的组合图像的像素进行操作。它促使生成图像具有空间连续性，从而避免结果过度像素化。你
可以将其理解为正则化损失。

- 总变差损失

In [0]:
def total_variation_loss(x):
  a = K.square(x[:, :img_height -1, :img_width -1, :] -
              x[:, 1:, :img_width -1, :])
  b = K.square(x[:, :img_height -1, :img_width -1, :] -
             x[:, :img_height -1, 1:, :])
  return K.sum(K.pow(a + b, 1.25))

我们需要最小化的损失是这三项损失的加权平均。为了计算内容损失，我们只使用一个靠
顶部的层，即block5_conv2 层；而对于风格损失，我们需要使用一系列层，既包括顶层也包
括底层。最后还需要添加总变差损失。
根据所使用的风格参考图像和内容图像，很可能还需要调节content_weight 系数（内容
损失对总损失的贡献比例）。更大的content_weight 表示目标内容更容易在生成图像中被识
别出来。

- 定义需要最小化的最终损失

In [17]:
#将层的名称映射为激活张量的字典
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
#用于内容损失的层
content_layer = 'block5_conv2'
#用于风格损失的层
style_layers = ['block1_conv1',
               'block2_conv1',
               'block3_conv1',
               'block4_conv1',
               'block5_conv1']

#损失分量的加权平均所使用的权重
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025

#添加内容损失
loss = K.variable(0.) #在定义损失时将所有分量添加到这个标量变量中
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(target_image_features, combination_features)


#添加每个目标层的风格损失分量
for layer_name in style_layers:
  layer_features = outputs_dict[layer_name]
  style_reference_features = layer_features[1, :, :, :]
  combination_features = layer_features[2, :, :, :]
  sl = style_loss(style_reference_features, combination_features)
  loss += (style_weight / len(style_layers)) * sl
  
#添加总变差损失  
loss += total_variation_weight * total_variation_loss(combination_image)

W0816 03:01:18.469416 140092940949376 variables.py:2429] Variable += will be deprecated. Use variable.assign_add if you want assignment to the variable value or 'x = x + y' if you want a new python Tensor object.


最后需要设置梯度下降过程。在Gatys 等人最初的论文中，使用L-BFGS 算法进行最优化，
所以我们这里也将使用这种方法。这是本例与8.2 节DeepDream 例子的主要区别。L-BFGS 算
法内置于SciPy 中，但SciPy 实现有两个小小的限制。

-  它需要将损失函数值和梯度值作为两个单独的函数传入。
-  它只能应用于展平的向量，而我们的数据是三维图像数组。

分别计算损失函数值和梯度值是很低效的，因为这么做会导致二者之间大量的冗余计算。
这一过程需要的时间几乎是联合计算二者所需时间的2 倍。为了避免这种情况，我们将创建一
个名为Evaluator 的Python 类，它可以同时计算损失值和梯度值，在第一次调用时会返回损
失值，同时缓存梯度值用于下一次调用。

- 设置梯度下降过程

In [18]:
grads = K.gradients(loss, combination_image)[0] #获取损失相对于生成图像的梯度

#用于获取当前损失值和当前梯度值的函数
fetch_loss_and_grads = K.function([combination_image],[loss,grads])

class Evaluator(object):
  #这个类将fetch_loss_and_grads 包
  #装起来，让你可以利用两个单独的方法
  #调用来获取损失和梯度，这是我们要使
  #用的SciPy 优化器所要求的
  
  def __init__(self):
    self.loss_value = None
    self.grads_values = None
    
  def loss(self, x):
    assert self.loss_value is None
    x = x.reshape((1, img_height, img_width, 3))
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    self.loss_value = loss_value
    self.grad_values = grad_values
    return self.loss_value
  
  def grads(self, x):
    assert self.loss_value is not None
    grad_values = np.copy(self.grad_values)
    self.loss_value = None
    self.grad_values = None
    return grad_values
  
  
evaluator = Evaluator()
    

W0816 03:15:38.448963 140092940949376 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


最后，可以使用SciPy 的L-BFGS 算法来运行梯度上升过程，在算法每一次迭代时都保存
当前的生成图像（这里一次迭代表示20 个梯度上升步骤）。

-  　风格迁移循环

In [0]:
from scipy.optimize import fmin_l_bfgs_b
import imageio
import time

In [28]:
localtime = time.localtime()
_,_,_,tm_hour,tm_min,*_ = localtime
result_prefix = 'my_result'
iterations = 20

#这是初始状态：目标图像
x = preprocess_image(target_image_path)
#将图像展平，因为scipy.optimize.fmin_l_bfgs_b 只能处理展平的向量
x = x.flatten()

for i in range(iterations):
  print('Start of iteration ', i)
  start_time = time.time()
  #对生成图像的像素运行
  #L-BFGS 最优化，以将神
  #经风格损失最小化。注意，
  #必须将计算损失的函数和
  #计算梯度的函数作为两个
  #单独的参数传入
  x, min_val, info = fmin_l_bfgs_b(evaluator.loss,
                                  x,
                                  fprime=evaluator.grads,
                                  maxfun=20)
  print('Current loss value:', min_val)
  
  #保存当前的生成图像
  img = x.copy().reshape((img_height, img_width, 3))
  img = deprocess_image(img)
  fname = result_prefix + '_at _iteration_%d_%d_%d.png' % (tm_hour,tm_min,i)
  imageio.imwrite(fname, img)
  print('Image saved as ',fname)
  end_time = time.time()
  print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration  0
Current loss value: 2047948800.0
Image saved as  my_result_at _iteration_5_49_0.png
Iteration 0 completed in 3s
Start of iteration  1
Current loss value: 1222187100.0
Image saved as  my_result_at _iteration_5_49_1.png
Iteration 1 completed in 3s
Start of iteration  2
Current loss value: 895021060.0
Image saved as  my_result_at _iteration_5_49_2.png
Iteration 2 completed in 3s
Start of iteration  3
Current loss value: 730217660.0
Image saved as  my_result_at _iteration_5_49_3.png
Iteration 3 completed in 3s
Start of iteration  4
Current loss value: 628251650.0
Image saved as  my_result_at _iteration_5_49_4.png
Iteration 4 completed in 3s
Start of iteration  5
Current loss value: 564356700.0
Image saved as  my_result_at _iteration_5_49_5.png
Iteration 5 completed in 3s
Start of iteration  6
Current loss value: 516258700.0
Image saved as  my_result_at _iteration_5_49_6.png
Iteration 6 completed in 3s
Start of iteration  7
Current loss value: 479371000.0
Image saved a